In [1]:
import sys
sys.path.append('../mydata/')
import scipy.io
import urllib.request
import dgl
import math
import numpy as np
import pandas as pd
from model import *
import torch
from data_loader import data_loader
from utils.data import load_data
from utils.pytorchtools import EarlyStopping
from utils.tools import evaluate_results_nc
import argparse
import time

def sp_to_spt(mat):
    coo = mat.tocoo()
    values = coo.data
    indices = np.vstack((coo.row, coo.col))

    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = coo.shape

    return torch.sparse.FloatTensor(i, v, torch.Size(shape))


def mat2tensor(mat):
    if type(mat) is np.ndarray:
        return torch.from_numpy(mat).type(torch.FloatTensor)
    return sp_to_spt(mat)

def sampler(anchor, sample):
    assert anchor.size(0) == sample.size(0)
    num_nodes = anchor.size(0)
    device = anchor.device
    pos_mask = torch.eye(num_nodes, dtype=torch.float32, device=device)
    neg_mask = 1. - pos_mask
    return anchor, sample, pos_mask, neg_mask

def _similarity(h1: torch.Tensor, h2: torch.Tensor):
    h1 = F.normalize(h1)
    h2 = F.normalize(h2)
    return h1 @ h2.t()

def InfoNCE_loss(anchor, sample, pos_mask, neg_mask, tau=0.2):
        sim = _similarity(anchor, sample) / tau
        exp_sim = torch.exp(sim) * (pos_mask + neg_mask)
        log_prob = sim - torch.log(exp_sim.sum(dim=1, keepdim=True))
        loss = log_prob * pos_mask
        loss = loss.sum(dim=1) / pos_mask.sum(dim=1)
        return -loss.mean()
    
def TripletMargin_loss(anchor, sample, pos_mask, neg_mask=None, margin = 1.0, p = 2.0):
        loss_fn = torch.nn.TripletMarginLoss(margin=margin, p=p, reduction='none')
        num_anchors = anchor.size()[0]
        num_samples = sample.size()[0]

        # Key idea here:
        #  (1) Use all possible triples (will be num_anchors * num_positives * num_negatives triples in total)
        #  (2) Use PyTorch's TripletMarginLoss to compute the marginal loss for each triple
        #  (3) Since TripletMarginLoss accepts input tensors of shape (B, D), where B is the batch size,
        #        we have to manually construct all triples and flatten them as an input tensor in the
        #        shape of (num_triples, D).
        #  (4) We first compute loss for all triples (including those that are not anchor - positive - negative), which
        #        will be num_anchors * num_samples * num_samples triples, and then filter them with masks.

        # compute negative mask
        neg_mask = 1. - pos_mask if neg_mask is None else neg_mask

        anchor = torch.unsqueeze(anchor, dim=1)  # [N, 1, D]
        anchor = torch.unsqueeze(anchor, dim=1)  # [N, 1, 1, D]
        anchor = anchor.expand(-1, num_samples, num_samples, -1)  # [N, M, M, D]
        anchor = torch.flatten(anchor, end_dim=1)  # [N * M * M, D]

        pos_sample = torch.unsqueeze(sample, dim=0)  # [1, M, D]
        pos_sample = torch.unsqueeze(pos_sample, dim=2)  # [1, M, 1, D]
        pos_sample = pos_sample.expand(num_anchors, -1, num_samples, -1)  # [N, M, M, D]
        pos_sample = torch.flatten(pos_sample, end_dim=1)  # [N * M * M, D]

        neg_sample = torch.unsqueeze(sample, dim=0)  # [1, M, D]
        neg_sample = torch.unsqueeze(neg_sample, dim=0)  # [1, 1, M, D]
        neg_sample = neg_sample.expand(num_anchors, -1, num_samples, -1)  # [N, M, M, D]
        neg_sample = torch.flatten(neg_sample, end_dim=1)  # [N * M * M, D]

        loss = loss_fn(anchor, pos_sample, neg_sample)  # [N, M, M]
        loss = loss.view(num_anchors, num_samples, num_samples)

        pos_mask1 = torch.unsqueeze(pos_mask, dim=2)  # [N, M, 1]
        pos_mask1 = pos_mask1.expand(-1, -1, num_samples)  # [N, M, M]
        neg_mask1 = torch.unsqueeze(neg_mask, dim=1)  # [N, 1, M]
        neg_mask1 = neg_mask1.expand(-1, num_samples, -1)  # [N, M, M]

        pair_mask = pos_mask1 * neg_mask1  # [N, M, M]
        num_pairs = pair_mask.sum()

        loss = loss * pair_mask
        loss = loss.sum()

        return loss / num_pairs

def build_heterograph(args):
    if args.device >= 0:
        device = torch.device("cuda:"+str(args.device))
    else:
        device = torch.device('cpu')
    
    print('loading data >>>>>>>>>>>>>>>>>>>>>')
    dataset = data_loader('../mydata/'+args.dataset)
    edge_dict = {}

    for i, meta_path in dataset.links['meta'].items():
        edge_dict[(str(meta_path[0]), str(meta_path[0]) + '_' + str(meta_path[1]), str(meta_path[1]))] = (torch.tensor(dataset.links['data'][i].tocoo().row - dataset.nodes['shift'][meta_path[0]]), torch.tensor(dataset.links['data'][i].tocoo().col - dataset.nodes['shift'][meta_path[1]]))

    node_count = {}
    for i, count in dataset.nodes['count'].items():
        print(i, node_count)
        node_count[str(i)] = count

    print('building dgl.heterograph >>>>>>>>>>>>>>>>>>>>>')
    G = dgl.heterograph(edge_dict, num_nodes_dict = node_count, device=device)
    """
    for ntype in G.ntypes:
        G.nodes[ntype].data['inp'] = dataset.nodes['attr'][ntype]
        # print(G.nodes['attr'][ntype].shape)
    """

    G.node_dict = {}
    G.edge_dict = {}
    for ntype in G.ntypes:
        G.node_dict[ntype] = len(G.node_dict)
    for etype in G.etypes:
        G.edge_dict[etype] = len(G.edge_dict)
        G.edges[etype].data['id'] = torch.ones(G.number_of_edges(etype), dtype=torch.long).to(device) * G.edge_dict[etype] 
    return G,dataset

def build_dataloader(G, sample_list, dst_dict, batch_size):
    sampler = dgl.dataloading.neighbor.MultiLayerNeighborSampler(sample_list)
    collator = dgl.dataloading.NodeCollator(G, dst_dict, sampler)
    dataloader = torch.utils.data.DataLoader(collator.dataset, collate_fn=collator.collate, batch_size=batch_size, shuffle=True, drop_last=False)
    return dataloader


Using backend: pytorch


In [2]:
## 定义各种方法、模型，可以跳过
### train和inference两个函数对同质图进行了改写

class HGTLayer(nn.Module):
    def __init__(self,
                 in_dim,
                 out_dim,
                 node_dict,
                 edge_dict,
                 n_heads,
                 dropout = 0.2,
                 use_norm = False,
                 out_key='',
                 is_change_fe = False):
        super(HGTLayer, self).__init__()

        self.in_dim        = in_dim
        self.out_dim       = out_dim
        self.node_dict     = node_dict
        self.edge_dict     = edge_dict
        self.num_types     = len(node_dict)
        self.num_relations = len(edge_dict)
        self.total_rel     = self.num_types * self.num_relations * self.num_types
        self.n_heads       = n_heads
        self.d_k           = out_dim // n_heads
        self.sqrt_dk       = math.sqrt(self.d_k)
        self.att           = None
        self.out_key = out_key

        self.k_linears   = nn.ModuleList()
        self.q_linears   = nn.ModuleList()
        self.v_linears   = nn.ModuleList()
        self.a_linears   = nn.ModuleList()
        self.norms       = nn.ModuleList()
        self.use_norm    = use_norm

        for t in range(self.num_types):
            self.k_linears.append(nn.Linear(in_dim,   out_dim))
            self.q_linears.append(nn.Linear(in_dim,   out_dim))
            self.v_linears.append(nn.Linear(in_dim,   out_dim))
            self.a_linears.append(nn.Linear(out_dim,  out_dim))
            if use_norm:
                self.norms.append(nn.LayerNorm(out_dim))

        self.relation_pri   = nn.Parameter(torch.ones(self.num_relations, self.n_heads))
        self.relation_att   = nn.Parameter(torch.Tensor(self.num_relations, n_heads, self.d_k, self.d_k))
        self.relation_msg   = nn.Parameter(torch.Tensor(self.num_relations, n_heads, self.d_k, self.d_k))
        self.skip           = nn.Parameter(torch.ones(self.num_types))
        self.drop           = nn.Dropout(dropout)
        self.my_alpha = torch.tensor(0.)
        
        self.is_change_fe = is_change_fe

        nn.init.xavier_uniform_(self.relation_att)
        nn.init.xavier_uniform_(self.relation_msg)

    def forward(self, G, h):
        with G.local_scope():
            node_dict, edge_dict = self.node_dict, self.edge_dict
            for srctype, etype, dsttype in G.canonical_etypes:
                sub_graph = G[srctype, etype, dsttype]

                k_linear = self.k_linears[node_dict[srctype]]
                v_linear = self.v_linears[node_dict[srctype]]
                q_linear = self.q_linears[node_dict[dsttype]]

                k = k_linear(h[srctype]).view(-1, self.n_heads, self.d_k)  ## 得到多头的key, query, value, size: (node_nums, n_heads, out_dim // n_heads)
                v = v_linear(h[srctype]).view(-1, self.n_heads, self.d_k)
                q = q_linear(h[dsttype][:G.number_of_dst_nodes(dsttype)]).view(-1, self.n_heads, self.d_k)

                e_id = self.edge_dict[etype]

                relation_att = self.relation_att[e_id]  ## size: (n_heads, out_dim // n_heads, out_dim // n_heads)
                relation_pri = self.relation_pri[e_id]
                relation_msg = self.relation_msg[e_id]

                k = torch.einsum("bij,ijk->bik", k, relation_att)
                v = torch.einsum("bij,ijk->bik", v, relation_msg)

                sub_graph.srcdata['k'] = k
                sub_graph.dstdata['q'] = q
                sub_graph.srcdata['v_%d' % e_id] = v

                sub_graph.apply_edges(fn.v_dot_u('q', 'k', 't'))  # 对q和k特征进行点乘，求kq值，准备求attention value
                attn_score = sub_graph.edata.pop('t').sum(-1) * relation_pri / self.sqrt_dk
                attn_score = edge_softmax(sub_graph, attn_score, norm_by='dst')

                sub_graph.edata['t'] = attn_score.unsqueeze(-1)   ## 得到每一种类型的attention value
            ## message passing along the eage type
            G.multi_update_all({etype : (fn.u_mul_e('v_%d' % e_id, 't', 'm'), fn.sum('m', 't')) \
                                for etype, e_id in edge_dict.items()}, cross_reducer = 'mean')
            new_h = {}
            '''
                Step 3: Target-specific Aggregation
                x = norm( W[node_type] * gelu( Agg(x) ) + x )
            '''
            
            for ntype in G.ntypes:
                n_id = node_dict[ntype]
                alpha = torch.sigmoid(self.skip[n_id])
                # print(alpha)
                t = G.dstnodes[ntype].data['t'].view(-1, self.out_dim)
                
                
                if self.is_change_fe:
                    dst_nodes = h[ntype][:G.number_of_dst_nodes(ntype)]
                    is_label = G.dstnodes[ntype].data['is_label'].view(-1,1)
#                     print('>>>>>>>>>>>>>>>>>is_label: ',is_label.shape)
#                     print('>>>>>>>>>>>>>>>>>h[ntype][:G.number_of_dst_nodes(ntype)]: ',h[ntype][:G.number_of_dst_nodes(ntype)].shape)
                    label_nodes = is_label*dst_nodes
                    unlabel_nodes = (1-is_label)*dst_nodes
                    unlabel_t = (1-is_label)*t
#                     print('unlabel_nodes: ',unlabel_nodes)
#                     print('unlabel_t: ',unlabel_t)
                    
                    unlabel_idx = (is_label == 0).nonzero()[:,0]
#                     print('is_label: ',is_label)
#                     print('unlabel_idx',unlabel_idx)
                    
                    delta = torch.abs(unlabel_t[unlabel_idx] - unlabel_nodes[unlabel_idx])
                    sorted_values,sorted_indexs = torch.sort(delta, dim=-1) #按行排序，sorted_values[:,0]得到邻居节点最相似的特征
            
                    dst_fe = torch.gather(unlabel_nodes[unlabel_idx],1,sorted_indexs[:,0].view(-1,1)) #按行取对应位置的特征：目标节点对应的特征
                    t_fe = torch.gather(t[unlabel_idx],1,sorted_indexs[:,0].view(-1,1)) #按行取对应位置的特征：目标节点对应的特征
#                     print('dst_fe: ',dst_fe)
#                     print('t_fe: ',t_fe)

                    # 替换目标节点的特征
                    # dim=1表示逐行进行列填充
                    unlabel_nodes[unlabel_idx] = unlabel_nodes[unlabel_idx].scatter(1, sorted_indexs[:,0].view(-1,1), t_fe.view(-1,1))
                    dst_nodes = label_nodes + unlabel_nodes
                    
                    # 替换邻居节点的特征
                    unlabel_t[unlabel_idx] = unlabel_t[unlabel_idx].scatter(1, sorted_indexs[:,0].view(-1,1), dst_fe.view(-1,1))
                    t = is_label*t + unlabel_t
                    trans_out = self.drop(self.a_linears[n_id](t))
#                     print('unlabel_nodes: ',unlabel_nodes)
#                     print('unlabel_t: ',unlabel_t)
                
                    trans_out = trans_out * alpha + dst_nodes * (1-alpha)
                
                else:
                    trans_out = self.drop(self.a_linears[n_id](t))

                    trans_out = trans_out * alpha + h[ntype][:G.number_of_dst_nodes(ntype)] * (1-alpha)
                # trans_out = trans_out * self.my_alpha + h[ntype][:G.number_of_dst_nodes(ntype)] * (1-self.my_alpha)
                if self.use_norm:
                    new_h[ntype] = self.norms[n_id](trans_out)
                else:
                    new_h[ntype] = trans_out
                
            return new_h

class HGT(nn.Module):
    def __init__(self, G, node_dict, edge_dict, n_inps, n_hid, n_out, n_layers, n_heads, use_norm = True, out_key='', use_t=False, dropout=0.2):
        super(HGT, self).__init__()
        self.node_dict = node_dict
        self.edge_dict = edge_dict
        
        ##kk
        self.gcs = nn.ModuleList()
        self.gcs2 = nn.ModuleList()
        ##kk
        
        self.n_inps = n_inps
        self.n_hid = n_hid
        self.n_out = n_out
        self.n_layers = n_layers
        self.adapt_ws  = nn.ModuleList()
        for t in range(len(node_dict)):
            self.adapt_ws.append(nn.Linear(self.n_inps[t], n_hid))
        
        ##kk
        for _ in range(n_layers):
            self.gcs.append(HGTLayer(n_hid, n_hid, node_dict, edge_dict, n_heads, dropout=dropout, use_norm = use_norm, out_key=out_key))
            self.gcs2.append(HGTLayer(n_hid, n_hid, node_dict, edge_dict, n_heads, dropout=dropout, use_norm = use_norm, out_key=out_key, is_change_fe=True))
        ##kk
        
        #self._in = nn.Linear(n_inp,   n_hid)
        self.out = nn.Linear(n_hid, n_out)
        self.log_softmax_out = nn.LogSoftmax(dim=1)
        self.softmax_out = nn.Softmax(dim=1)
        self.use_t = use_t
#         self.RTE = RelTemporalEncoding(n_inp, max_len=367)

    def forward(self, block, out_key):
        h = {}
        h2 = {}
        for b in range(len(block)):
            if b == 0:
                h = {}
                h2 = {}
                for ntype in block[b].ntypes:   ## 这里写入模型输入
                    n_id = self.node_dict[ntype]
                    if self.use_t:
                        inp_emd = self.RTE(block[b].srcnodes[ntype].data['inp'], block[b].srcnodes[ntype].data['del_t'])
                        h[ntype] = F.gelu(self.adapt_ws[n_id](inp_emd))
                        h2[ntype] = F.gelu(self.adapt_ws[n_id](inp_emd))
                    else:
                        try:
                            h[ntype] = F.gelu(self.adapt_ws[n_id](block[b].srcnodes[ntype].data['inp']))
                            h2[ntype] = F.gelu(self.adapt_ws[n_id](block[b].srcnodes[ntype].data['inp']))
                        except:
                            h[ntype] = F.gelu(self.adapt_ws[n_id](block[b].srcnodes[ntype].data['inp'].float()))
                            h2[ntype] = F.gelu(self.adapt_ws[n_id](block[b].srcnodes[ntype].data['inp'].float()))
#             print('>>>>>>>>>>>>>>>>>h2: ',h2['_N'].shape)
            h = self.gcs[b](block[b], h) ## 表示有多少层HGT，层数越多，跳数越多
            h2 = self.gcs2[b](block[b], h2)
        return self.softmax_out(self.out(h[out_key])), self.softmax_out(self.out(h2[out_key])),h[out_key],h2[out_key]


In [3]:

def run_model_DBLP(args, G, rep):
    if args.device >= 0:
        device = torch.device("cuda:"+str(args.device))
    else:
        device = torch.device('cpu')
    
    feats_type = args.feats_type
    features_list, adjM, labels, train_val_test_idx, dl = load_data(args.dataset)
    features_list = [mat2tensor(features).to(device) for features in features_list]

    if feats_type == 0:
        in_dims = [features.shape[1] for features in features_list]
    elif feats_type == 1 or feats_type == 5:
        save = 0 if feats_type == 1 else 2
        in_dims = []#[features_list[0].shape[1]] + [10] * (len(features_list) - 1)
        for i in range(0, len(features_list)):
            if i == save:
                in_dims.append(features_list[i].shape[1])
            else:
                in_dims.append(10)
                features_list[i] = torch.zeros((features_list[i].shape[0], 10)).to(device)
    elif feats_type == 2 or feats_type == 4:
        save = feats_type - 2
        in_dims = [features.shape[0] for features in features_list]
        for i in range(0, len(features_list)):
            if i == save:
                in_dims[i] = features_list[i].shape[1]
                continue
            dim = features_list[i].shape[0]
            indices = np.vstack((np.arange(dim), np.arange(dim)))
            indices = torch.LongTensor(indices)
            values = torch.FloatTensor(np.ones(dim))
            features_list[i] = torch.sparse.FloatTensor(indices, values, torch.Size([dim, dim])).to(device)
    elif feats_type == 3:
        in_dims = [features.shape[0] for features in features_list]
        for i in range(len(features_list)):
            dim = features_list[i].shape[0]
            indices = np.vstack((np.arange(dim), np.arange(dim)))
            indices = torch.LongTensor(indices)
            values = torch.FloatTensor(np.ones(dim))
            features_list[i] = torch.sparse.FloatTensor(indices, values, torch.Size([dim, dim])).to(device)

    ## 添加图特征
    for ntype in G.ntypes:
        G.nodes[ntype].data['inp'] = features_list[int(ntype)]#.to(device)
    labels = torch.LongTensor(labels).to(device)
    train_idx = train_val_test_idx['train_idx']
    train_idx = np.sort(train_idx)
    val_idx = train_val_test_idx['val_idx']
    val_idx = np.sort(val_idx)
    test_idx = train_val_test_idx['test_idx']
    test_idx = np.sort(test_idx)
    unlabel_idx = train_val_test_idx['unlabel_idx']
    unlabel_idx = np.sort(unlabel_idx)
    
    ## kk
    is_label = torch.ones(len(G.nodes['0'].data['inp'])).view(-1,1)
    is_label[unlabel_idx]=0
    for ntype in G.ntypes:
        if ntype == '0':
            G.nodes[ntype].data['is_label'] = is_label.to(device)
            G.nodes[ntype].data['label'] = labels
        else:
            G.nodes[ntype].data['is_label'] = torch.zeros(len(G.nodes[ntype].data['inp'])).view(-1,1).to(device)
            G.nodes[ntype].data['label'] = torch.zeros(len(G.nodes[ntype].data['inp'])).view(-1,1).to(device)
    ## kk

    print(len(train_idx), len(test_idx), len(val_idx), len(unlabel_idx))
    model = HGT(G, G.node_dict, G.edge_dict, n_inps=in_dims, n_hid=args.hidden_dim, n_out=labels.max().item()+1, n_layers=args.num_layers, n_heads=args.num_heads, use_norm = args.use_norm, out_key='0', use_t=False, dropout=args.dropout).to(device)

    optimizer = torch.optim.AdamW(model.parameters(), weight_decay=args.weight_decay)#, lr=args.lr, weight_decay=args.weight_decay)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, total_steps=args.schedule_step, max_lr = 1e-3, pct_start=0.05)

    early_stopping = EarlyStopping(patience=args.patience, verbose=True, save_path='../HGT/checkpoint/checkpoint_batch_{}_{}l_{}th.pt'.format(args.dataset, args.num_layers, rep))
    train_step = 0
    
    train_dst_dict = {'0': np.array(list(train_idx)+list(unlabel_idx))}
    val_dst_dict = {'0': val_idx}
    test_dst_dict = {'0': test_idx}
    for epoch in range(args.epoch):
        dataloader = build_dataloader(G.cpu(), sample_list, train_dst_dict, args.batch_size)
        
        for input_nodes, output_nodes, blocks in dataloader:
            blocks = [b.to(device) for b in blocks]
            print('blocks: ',len(dataloader))
#             print('input_nodes : ', input_nodes)
            is_label = torch.squeeze(blocks[-1].dstdata['is_label']['0'],1)  ##新增一列标记数据是否为有标签数据
            batch_train_idx = (is_label == 1).nonzero().view(-1,1)  
            batch_unlabels_idx = (is_label == 0).nonzero().view(-1,1)
            
            with torch.autograd.set_detect_anomaly(True):
                t_start = time.time()
                model.train()
                # , embeddings2 
                logits1, logits2, embeddings1, embeddings2 = model(blocks, '0')
                
                
                if not len(batch_train_idx):
                    anchor, sample, pos_mask, neg_mask = sampler(anchor=torch.squeeze(logits1[batch_unlabels_idx],1), sample=torch.squeeze(logits2[batch_unlabels_idx],1))
                    loss_ul = TripletMargin_loss(anchor, sample, pos_mask)
                    train_loss = loss_ul
                if not len(batch_unlabels_idx):
                    train_loss = F.cross_entropy(torch.squeeze(logits1[batch_train_idx],1), torch.squeeze(blocks[-1].dstdata['label']['0'][batch_train_idx],1))
                if len(batch_train_idx) and len(batch_unlabels_idx):
                    anchor, sample, pos_mask, neg_mask = sampler(anchor=torch.squeeze(logits1[batch_unlabels_idx],1), sample=torch.squeeze(logits2[batch_unlabels_idx],1))
                    loss_ul = TripletMargin_loss(anchor, sample, pos_mask)
                    train_loss = (1-args.rate)*F.cross_entropy(torch.squeeze(logits1[batch_train_idx],1), torch.squeeze(blocks[-1].dstdata['label']['0'][batch_train_idx],1)) + args.rate*loss_ul
                
                print('cross_entropy>>>>>>>> ',F.cross_entropy(torch.squeeze(logits1[batch_train_idx],1), torch.squeeze(blocks[-1].dstdata['label']['0'][batch_train_idx],1)))
                print('mse_loss>>>>>>>> ',F.mse_loss(torch.squeeze(logits1[batch_unlabels_idx],1),torch.squeeze(logits2[batch_unlabels_idx],1)))
                print('infonce_loss>>>>>>>> ',loss_ul)

                
                pred = logits1.argmax(1)
               
                train_acc = (pred[batch_train_idx] == blocks[-1].dstdata['label']['0'][batch_train_idx]).float().mean()


                optimizer.zero_grad()
                train_loss.backward()
                optimizer.step()
                train_step += 1
                scheduler.step(train_step)
                t_end = time.time()
            print('Epoch {:03d} | Train_Step: {:03d} | Train_Loss: {:.4f} | Train_acc: {:.4f} |Time: {:.4f}'.format(epoch, train_step, train_loss.item(), train_acc.item(), t_end-t_start))
            t_start = time.time()
            model.eval()
            with torch.no_grad():
                dataloader_val = build_dataloader(G.cpu(), sample_list, val_dst_dict, args.batch_size)
                
                val_acc_list = []
                val_len_list = []
                val_loss_list = []
                for input_nodes0, output_nodes0, blocks0 in dataloader_val: 
                    blocks0 = [b.to(device) for b in blocks0]
                    logits1, logits2, val_embeddings1, val_embeddings2 = model(blocks0, '0')

                    val_loss = F.cross_entropy(torch.squeeze(logits1,1), blocks0[-1].dstdata['label']['0'])
                    pred = logits1.cpu().numpy().argmax(axis=1)
                    pred2 = logits2.cpu().numpy().argmax(axis=1)
                    val_acc_tmp = (pred == blocks0[-1].dstdata['label']['0'].cpu().numpy()).sum()
                    val_acc_list.append(val_acc_tmp)
                    val_len_list.append(len(pred))
                    
                    val_loss_list.append(val_loss.cpu().numpy())
                print('val_loss_list',val_loss_list)
                val_acc = sum(val_acc_list)/sum(val_len_list)
                val_loss = sum(val_loss_list)/len(val_loss_list)

                ######### test ###############
                dataloader_test = build_dataloader(G.cpu(), sample_list, test_dst_dict, args.batch_size)
                
                test_acc_list1,test_acc_list2 = [],[]
                test_len_list = []
                for input_nodes1, output_nodes1, blocks1 in dataloader_test: 
                    blocks1 = [b.to(device) for b in blocks1]
                    logits1, logits2, _, _ = model(blocks1, '0')

                    pred = logits1.cpu().numpy().argmax(axis=1)
                    pred2 = logits2.cpu().numpy().argmax(axis=1)
                    test_acc_tmp = (pred == blocks1[-1].dstdata['label']['0'].cpu().numpy()).sum()
                    test_acc_list1.append(test_acc_tmp)
                    test_len_list.append(len(pred))
                    test_acc_tmp = (pred2 == blocks1[-1].dstdata['label']['0'].cpu().numpy()).sum()
                    test_acc_list2.append(test_acc_tmp)
                test_acc1 = sum(test_acc_list1)/sum(test_len_list)
                test_acc2 = sum(test_acc_list2)/sum(test_len_list)
                print('test_acc2',test_acc2)
                
                t_end = time.time()
                print('Epoch {:05d} | Val_Loss {:.4f} | Val_acc {:.4f} | Test_acc {:.4f} | Time(s) {:.4f}'.format(
                    epoch, val_loss.item(), val_acc.item(), test_acc1.item(), t_end - t_start))
                early_stopping(torch.tensor(val_loss), model)
                if early_stopping.early_stop:
                    print('Early stopping!')
                    return model,test_idx
    return model,test_idx
  
def inference(args, model, test_idx, dataset, G, rep):
    test_dst_dict = {'0': test_idx}
    if args.device >= 0:
        device = torch.device("cuda:"+str(args.device))
    else:
        device = torch.device('cpu')
    
    model.load_state_dict(torch.load('../HGT/checkpoint/checkpoint_batch_{}_{}l_{}th.pt'.format(args.dataset, args.num_layers, rep)))
    model.eval()
#     test_logits = []
    with torch.no_grad():
        dataloader_test = build_dataloader(G.cpu(), sample_list, test_dst_dict, args.batch_size)
                
        test_emb = torch.tensor([])
        for input_nodes1, output_nodes1, blocks1 in dataloader_test: 
            blocks1 = [b.to(device) for b in blocks1]
            logits1, _, test_embeddings, _ = model(blocks1, '0')
            test_label = blocks1[-1].dstdata['label']['0'].cpu().numpy()

            if not len(test_emb):
                test_emb = test_embeddings
                truth_label = test_label.tolist()
            else:
                test_emb = torch.cat([test_emb, test_embeddings], dim=0)
                truth_label += test_label.tolist()
        svm_macro_f1_list, svm_micro_f1_list, nmi_mean, nmi_std, ari_mean, ari_std = evaluate_results_nc(
                test_emb.cpu(), np.array(truth_label), num_classes=dataset.labels_test['num_classes'])

        return svm_macro_f1_list, svm_micro_f1_list, nmi_mean, nmi_std, ari_mean, ari_std

In [4]:

# if __name__ == '__main__':
ap = argparse.ArgumentParser(description='MRGNN testing for the DBLP dataset')
ap.add_argument('--feats-type', type=int, default=1,
                help='Type of the node features used. ' +
                        '0 - loaded features; ' +
                        '1 - only target node features (zero vec for others); ' +
                        '2 - only target node features (id vec for others); ' +
                        '3 - all id vec. Default is 2;' +
                    '4 - only term features (id vec for others);' + 
                    '5 - only term features (zero vec for others).')
ap.add_argument('--hidden_dim', type=int, default=64, help='Dimension of the node hidden state. Default is 64.')
ap.add_argument('--num_heads', type=int, default=8, help='Number of the attention heads. Default is 8.')
ap.add_argument('--epoch', type=int, default=300, help='Number of epochs.')
ap.add_argument('--patience', type=int, default=30, help='Patience.')
ap.add_argument('--repeat', type=int, default=5, help='Repeat the training and testing for N times. Default is 1.')
ap.add_argument('--num_layers', type=int, default=8) #5
ap.add_argument('--lr', type=float, default=1e-2) #5e-3
ap.add_argument('--dropout', type=float, default=0)
ap.add_argument('--weight-decay', type=float, default=1e-3) #1e-4
ap.add_argument('--slope', type=float, default=0.05)
ap.add_argument('--dataset', type=str, default='DBLP')  # Freebase
ap.add_argument('--edge-feats', type=int, default=64)
ap.add_argument('--device', type=int, default=0)
ap.add_argument('--schedule_step', type=int, default=300)
ap.add_argument('--use_norm', type=bool, default=True)
ap.add_argument('--rate', type=float, default=0.2)
ap.add_argument('--batch_size', type=float, default=1024)
ap.add_argument('--sample_nb', type=int, default=1000)

args = ap.parse_args(args=[])
g, dataset = build_heterograph(args)

sample_list = []
sample_dict = {}
for i in range(args.num_layers):
    for _, meta_path in dataset.links['meta'].items():
        sample_dict[(str(meta_path[0]), str(meta_path[0]) + '_' + str(meta_path[1]), str(meta_path[1]))] = args.sample_nb
    sample_list.append(sample_dict)

micro,macro = [],[]
svm_macro_f1_lists,svm_micro_f1_lists,nmi_mean_list,nmi_std_list,ari_mean_list,ari_std_list = [],[],[],[],[],[]
for r in range(args.repeat):
    print('running the '+str(r)+' times >>>>>>>>>>>>')
#     model,test_idx,dataset,dl = run_model_DBLP(args, g, dataset, r, labels_test1)
    model,test_idx = run_model_DBLP(args, g, r)
    eval_res,svm_macro_f1_list, svm_micro_f1_list, nmi_mean, nmi_std, ari_mean, ari_std = inference(args, model, test_idx, dataset, g, r)
    macro.append(eval_res['macro-f1'])
    micro.append(eval_res['micro-f1'])
    svm_macro_f1_lists.append(svm_macro_f1_list)
    svm_micro_f1_lists.append(svm_micro_f1_list)
    nmi_mean_list.append(nmi_mean)
    nmi_std_list.append(nmi_std)
    ari_mean_list.append(ari_mean)
    ari_std_list.append(ari_std)

    
m_macro = np.mean(macro)
std_macro = np.std(macro)
m_micro = np.mean(micro)
std_micro = np.std(micro)
sqrt_n = np.sqrt(len(micro))
print('------------------------------ final result ----------------------------------------')
print('final result macro_f1 >>>>>>>>>> ',m_macro,'+-',1.96*std_macro/sqrt_n)
print('final result micro_f1 >>>>>>>>>> ',m_micro,'+-',1.96*std_micro/sqrt_n)

# print out a summary of the evaluations
svm_macro_f1_lists = np.transpose(np.array(svm_macro_f1_lists), (1, 0, 2))
svm_micro_f1_lists = np.transpose(np.array(svm_micro_f1_lists), (1, 0, 2))
nmi_mean_list = np.array(nmi_mean_list)
nmi_std_list = np.array(nmi_std_list)
ari_mean_list = np.array(ari_mean_list)
ari_std_list = np.array(ari_std_list)
print('----------------------------------------------------------------')
print('SVM tests summary')
print('Macro-F1: ' + ', '.join(['{:.6f}~{:.6f} ({:.1f})'.format(
    macro_f1[:, 0].mean(), macro_f1[:, 1].mean(), train_size) for macro_f1, train_size in
    zip(svm_macro_f1_lists, [0.8, 0.6, 0.4, 0.2])]))
print('Micro-F1: ' + ', '.join(['{:.6f}~{:.6f} ({:.1f})'.format(
    micro_f1[:, 0].mean(), micro_f1[:, 1].mean(), train_size) for micro_f1, train_size in
    zip(svm_micro_f1_lists, [0.8, 0.6, 0.4, 0.2])]))
print('K-means tests summary')
print('NMI: {:.6f}~{:.6f}'.format(nmi_mean_list.mean(), nmi_std_list.mean()))
print('ARI: {:.6f}~{:.6f}'.format(ari_mean_list.mean(), ari_std_list.mean()))


loading data >>>>>>>>>>>>>>>>>>>>>
0 {}
1 {'0': 4057}
2 {'0': 4057, '1': 14328}
3 {'0': 4057, '1': 14328, '2': 7723}
building dgl.heterograph >>>>>>>>>>>>>>>>>>>>>
running the 0 times >>>>>>>>>>>>
975 2839 243 2839
blocks:  4


RuntimeError: The size of tensor a (582169) must match the size of tensor b (763) at non-singleton dimension 0

In [ ]:

import pandas as pd
from data_loader import gen_file_for_evaluate
data = 'DBLP'

for i in range(5):
    df = pd.read_csv('../result/'+data+'_'+str(i)+'.txt', sep='\t', names=["node_id","node_name","node_type","label"])
    df[['label']].to_csv('../result1/'+data+'_'+str(i)+'.txt', sep='\t',index=False, header = False)

In [9]:
test_dst_dict = {'0': test_idx}
svm_macro_f1_list, svm_micro_f1_list, nmi_mean, nmi_std, ari_mean, ari_std = inference(args, model, test_idx, dataset, g, r)


SVM test
Macro-F1: 0.837679~0.012461 (0.8), 0.841881~0.011881 (0.6), 0.836755~0.007073 (0.4), 0.805841~0.009623 (0.2)
Micro-F1: 0.849120~0.010826 (0.8), 0.852025~0.011701 (0.6), 0.846890~0.007117 (0.4), 0.817430~0.010131 (0.2)
K-means test
NMI: 0.488869~0.001338
ARI: 0.540702~0.001362


ValueError: not enough values to unpack (expected 7, got 6)

In [10]:
1

1